# 自动微分

深度学习框架通过自动计算导数，即**自动微分(automatic differentiation)**加快求导

根据模型系统会构建**计算图(computational graph)**，跟踪哪些数据通过哪些操作组合产生输出

自动微分能够**反向传播(backpropagate)**梯度：跟踪计算图，填充每个参数的偏导数

## 简单例子

<img src="img/1-5/1.png" alt="例子" width="300">

In [1]:
import torch

x = torch.arange(4.0)
x

tensor([0., 1., 2., 3.])

计算y关于x的梯度之前需要一个地方存储梯度
不会每次对一个参数求导时都分配新内存，因为经常会成千上万次地更新相同的参数

In [2]:
x.requires_grad_(True)  # 等价x = torch.arange(4.0, requires_grad=True)
x.grad # 默认为None

In [3]:
# 计算y

y = 2 * torch.dot(x,x)
y

tensor(28., grad_fn=<MulBackward0>)

In [4]:
# 通过调用反向传播函数自动计算y关于每个分量的梯度

y.backward()
x.grad

tensor([ 0.,  4.,  8., 12.])

In [5]:
# 根据上一章梯度的性质可以的到y的梯度为4x
# 对比可知计算正确
x.grad == 4*x

tensor([True, True, True, True])

## 非标量变量的反向传播

当y不是标量时，向量y关于向量x的导数的最自然解释是一个矩阵
对于高阶和高维的y和x，求导的结果可以是一个高阶张量

In [16]:
# 非标量调用backward需要传入一个gradient参数，指定微分函数关于self的梯度
# 本例求偏导数之和，传递1是合适的
x.grad.zero_()  # 清空梯度
y = x * x
y.sum().backward() # 等价 y.backward(torch.ones(len(x)))
x.grad

tensor([0., 2., 4., 6.])

gradient参数的作用：**权重向量**，指定y中每个分量的权重
  - [1, 1, 1]：f(y) = 1*y1 + 1*y2 + 1*y3 = sum(y)
  - [2, 3, 4]：f(y) = 2*y1 + 3*y2 + 4*y3

In [15]:
x.grad.zero_()
y = x*x
y.backward(torch.tensor([1,2,3,4])) 
# 1*y1 + 2*y2 + 3*y3 + 4*y4
# 梯度为 [2*x1, 4*x2, 6*x3, 8*x4]
x.grad

tensor([ 0.,  4., 12., 24.])

## 分离计算

有时，我们希望将某些计算移动到记录的计算图之外

例如，假设y是作为x的函数计算的，而z则是作为y和x的函数计算的。我们想计算z关于x的梯度，但由于某种原因，希望将y视为一个常数， 并且只考虑到x在y被计算后发挥的作用

可以分离y来返回一个新变量u，该变量与y具有相同的值， 但丢弃计算图中如何计算y的任何信息

In [10]:
x.grad.zero_()

# 计算z=u*x关于x的偏导数
# u被作为常数，不是计算z=x*x*x关于x的偏导数
y = x*x
u = y.detach()
z = u*x

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

由于记录了y的计算结果，可以在y上调用反向传播，得到y关于x的导数，即2x

In [11]:
x.grad.zero_()
y.sum().backward()
x.grad == 2*x

tensor([True, True, True, True])

## python控制流的梯度计算

自动微分的好处：即使构建函数的计算图需要通过控制流(条件、循环、函数调用)，仍然可以计算梯度

In [12]:
def f(a):
    b = 2*a
    while b.norm() < 1000:
        b = b*2
    if b.sum() > 0:
        c = b
    else:
        c = 100*b
    return c

a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

分析f(a)：该函数时分段线性的，对于任何a，存在常量使得f(a)=ka，梯度为k，即d/a

In [13]:
a.grad == d/a

tensor(True)